# Pumped-Storage Optimisation with Genetic Algorithm and MILP

In [1]:
import pandas as pd
import numpy as np
import plotnine as pn
import plotly.graph_objs as go
import plotly.express as px
from tqdm.notebook import tqdm
from IPython.display import clear_output, display
import os
from itertools import product
import datetime

background_colour = "#F2F2F2"
pn.theme_set(
    pn.theme_classic()
    + pn.theme(
        text=pn.element_text(family="monospace"),
        plot_background=pn.element_rect(
            fill=background_colour, colour=background_colour
        ),
        panel_background=pn.element_rect(
            fill=background_colour, colour=background_colour
        ),
        legend_background=pn.element_rect(
            fill=background_colour, colour=background_colour
        ),
    )
)

%load_ext blackcellmagic

## Tuning

### Elite

In [6]:
tuning_history = pd.read_csv("./Tuning Results/202404120310_GA_Elite.csv")

combinations = (
    tuning_history.filter(regex="config/")
    .drop_duplicates(subset=[col for col in tuning_history.columns if "config/" in col])
    .reset_index(drop=True)
    .assign(id=lambda x: np.arange(1, x.shape[0] + 1))
)

tuning_history = tuning_history.merge(
    combinations, on=[col for col in tuning_history.columns if "config/" in col]
)

tuning_history_aggregated = (
    tuning_history.groupby(["id", "training_iteration"])
    .agg(fitness=("fitness", "mean"))
    .reset_index()
)

# Only keep the top combinations
num_best = 10
top_combinations = tuning_history_aggregated.groupby("id").last().reset_index().sort_values("fitness", ascending=False).head(num_best)["id"].to_list()
tuning_history_aggregated_top = tuning_history_aggregated.query("id in @top_combinations")

In [7]:
fig = px.line(
    data_frame=tuning_history_aggregated_top,
    x="training_iteration",
    y="fitness",
    color="id",
)
fig.show()

ID 79 looks very well behaved, let's take that one.

In [8]:
combinations.query("id == 79")

,config/MUTPB,config/POP_SIZE,config/INITIAL_MUTATION_RATE,config/FINAL_MUTATION_RATE,config/INITIAL_EXPLORATION,config/ELITISM,id
78,1,200,0.05,0.01,0.66,0.1,79


### Tournament

In [2]:
tuning_history = pd.read_csv("./Tuning Results/202404120811_GA_Tournament.csv")

combinations = (
    tuning_history.filter(regex="config/")
    .drop_duplicates(subset=[col for col in tuning_history.columns if "config/" in col])
    .reset_index(drop=True)
    .assign(id=lambda x: np.arange(1, x.shape[0] + 1))
)

tuning_history = tuning_history.merge(
    combinations, on=[col for col in tuning_history.columns if "config/" in col]
)

tuning_history_aggregated = (
    tuning_history.groupby(["id", "training_iteration"])
    .agg(fitness=("fitness", "mean"))
    .reset_index()
)

# Only keep the top combinations
num_best = 10
top_combinations = tuning_history_aggregated.groupby("id").last().reset_index().sort_values("fitness", ascending=False).head(num_best)["id"].to_list()
tuning_history_aggregated_top = tuning_history_aggregated.query("id in @top_combinations")

In [3]:
fig = px.line(
    data_frame=tuning_history_aggregated_top,
    x="training_iteration",
    y="fitness",
    color="id",
)
fig.show()

ID 81 looks very well behaved, let's take that one.

In [4]:
combinations.query("id == 81")

,config/MUTPB,config/POP_SIZE,config/CXPB,config/INITIAL_MUTATION_RATE,config/FINAL_MUTATION_RATE,config/INITIAL_EXPLORATION,config/TOURNAMENT_SIZE,id
80,1,200,0.65,0.05,0.01,0.66,23,81


## Training